# Text Model Augmentation with Markov Chains
- based on work by Marco Cerly and a blog post on towards data science blog (https://towardsdatascience.com/text-data-augmentation-makes-your-model-stronger-7232bd23704)

This will take a data set that exists and generate fake data based on what exists.

In [1]:
import os
import tqdm
import string
import random
import numpy as np
import pandas as pd
from collections import *
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import recall_score, precision_score, roc_auc_score, f1_score

ModuleNotFoundError: No module named 'sklearn'

In [1]:
def build_chain(texts):
    
    index = 1
    chain = {}
    
    for text in texts:
        
        text = text.split()
        for word in text[index:]:
            key = text[index-1]
            if key in chain:
                chain[key].append(word)
            else:
                chain[key] = [word]
            index += 1
        
        index = 1
    
    return chain
def create_sentence(chain, lenght):
    
    start = random.choice(list(chain.keys()))
    text = [start]
    while len(text) < lenght:
        try:
            after = random.choice(chain[start])
            start = after
            text.append(after)
        except: #end of the sentence
            #text.append('.')
            start = random.choice(list(chain.keys()))
    
    return ' '.join(text)
def Generator(x_train, y_train, rep, concat=False, seed=33):
    
    np.random.seed(seed)
    
    new_corpus, new_labels = [], []
    
    for i,lab in enumerate(np.unique(y_train)):
        selected = x_train[y_train == lab]
        chain = build_chain(selected)
        sentences = []
        for i in range(rep):
            lenght = int(np.random.choice(lenghts, 1, p=freq))
            sentences.append(create_sentence(chain, lenght))
        new_corpus.extend(sentences)
        new_labels.extend([lab]*rep)
    
    if concat:
        return list(x_train)+new_corpus, list(y_train)+new_labels
    
    return new_corpus, new_labels